In [5]:
import json
import random
import requests
import os

# --- Configuration ---
OPENROUTER_API_KEY = "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"  # Replace with your OpenRouter API key
OPENROUTER_API_URL = "https://openrouter.ai/api/v1/chat/completions"  # OpenRouter endpoint
JSON_FILE_PATH = "/content/yo.json"  # Path for the uploaded JSON file

# --- Load the JSON file with dishes ---
def load_dishes():
    """Loads the dish names from the JSON file."""
    if not os.path.exists(JSON_FILE_PATH):
        print(f"❌ Dish file not found at '{JSON_FILE_PATH}'. Please upload the correct file.")
        return []

    try:
        with open(JSON_FILE_PATH, 'r') as file:
            recipes = json.load(file)
        print(f"✅ Loaded {len(recipes)} dishes into the bot.")
        return recipes
    except json.JSONDecodeError:
        print("❌ Error reading the JSON file. Please check the file format.")
        return []

# --- Find recipe with case-insensitive and partial matching ---
def find_recipe(dish_name, recipe_dict):
    # Exact match
    if dish_name in recipe_dict:
        return recipe_dict[dish_name]

    # Partial match
    matches = [recipe for name, recipe in recipe_dict.items() if dish_name in name]
    return matches[0] if matches else None

# --- Call OpenRouter API for RAG-based recipe generation ---
def get_recipe_details(recipe):
    name = recipe.get('name', 'Unknown Dish')

    prompt = f"""
You are a culinary expert. Provide a detailed recipe for the following Maharashtrian dish:

Dish Name: {name}

Include:
- Step-by-step instructions
- Serving suggestions
- Cultural significance (if any)
- Nutritional notes
"""

    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "openai/gpt-4o",
        "messages": [
            {"role": "system", "content": "You are a helpful recipe assistant."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 1000,
        "temperature": 0.7
    }

    try:
        response = requests.post(OPENROUTER_API_URL, headers=headers, json=payload)
        response.raise_for_status()
        response_data = response.json()

        if response_data.get('choices') and response_data['choices'][0].get('message'):
            return response_data['choices'][0]['message']['content'].strip()
        elif response_data.get('error'):
            return f"❌ API Error: {response_data['error'].get('message', 'Unknown error')}"
        else:
            return "❌ The API did not return a valid response. Please check your API key, model access, or try again later."

    except requests.exceptions.HTTPError as http_err:
        return f"❌ HTTP Error: {http_err}"
    except requests.exceptions.RequestException as req_err:
        return f"❌ Request Error: {req_err}"
    except Exception as e:
        return f"❌ Unexpected Error: {e}"

# --- Main function ---
def main():
    print("""
🍽️ Welcome to the Maharashtrian Recipe Recommendation Bot! 🇮🇳
This bot uses Retrieval-Augmented Generation (RAG) to provide detailed recipes.
It contains popular Maharashtrian dishes. Enter the name of a dish to get its recipe.

Examples:
🥣 Breakfast: Poha, Thalipeeth
🥪 Snacks: Vada Pav, Misal Pav
🍛 Main Courses: Zunka Bhakri, Bharli Vangi
🍬 Desserts: Puran Poli, Modak

Type 'quit' to exit.
""")

    recipes = load_dishes()
    if not recipes:
        print("⚠️ Cannot proceed without loaded dishes.")
        return

    recipe_dict = {recipe['name'].lower(): recipe for recipe in recipes}

    while True:
        dish_name = input("\n🍽️ Enter a dish name (or type 'quit' to exit): ").strip().lower()

        if dish_name == 'quit':
            print("👋 Thank you for using the bot. Goodbye!")
            break

        recipe = find_recipe(dish_name, recipe_dict)
        if recipe:
            print(f"\n🔎 Found: {recipe['name']}\nFetching recipe... Please wait.")
            recipe_details = get_recipe_details(recipe)
            print(f"\n🍲 Recipe for {recipe['name']}\n{'-'*50}\n{recipe_details}\n")
        else:
            print("❌ Recipe not found. Try another name.")

# --- Run the bot ---
if __name__ == "__main__":
    main()



🍽️ Welcome to the Maharashtrian Recipe Recommendation Bot! 🇮🇳
This bot uses Retrieval-Augmented Generation (RAG) to provide detailed recipes.
It contains popular Maharashtrian dishes. Enter the name of a dish to get its recipe.

Examples:
🥣 Breakfast: Poha, Thalipeeth
🥪 Snacks: Vada Pav, Misal Pav
🍛 Main Courses: Zunka Bhakri, Bharli Vangi
🍬 Desserts: Puran Poli, Modak

Type 'quit' to exit.

✅ Loaded 1010 dishes into the bot.

🍽️ Enter a dish name (or type 'quit' to exit): poha

🔎 Found: Poha
Fetching recipe... Please wait.

🍲 Recipe for Poha
--------------------------------------------------
### Poha Recipe

**Poha** is a popular Maharashtrian breakfast dish made from flattened rice, known for its light yet flavorful profile. It is quick to prepare and loved for its simplicity and taste. Below is a detailed recipe for making Poha.

#### Ingredients

- 2 cups thick poha (flattened rice)
- 1 medium onion, finely chopped
- 1-2 green chilies, finely chopped
- 1/4 cup peanuts
- 1/2 teaspo